# Setting up the environment. PLEASE WAIT 🙃

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!pip install rembg[gpu] gradio==3.45.0

In [ ]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install

In [ ]:
# Install library
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [ ]:
ver_openpose = "v1.7.0"
! echo $ver_openpose

In [ ]:
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

In [ ]:
# manually downloading openpose models
%%bash
gdown 1I4UWTXN5RjtmcmUy6oKUzdUAfD1haNYC
unzip models.zip
mv /content/models/face/pose_iter_116000.caffemodel /content/openpose/models/face/pose_iter_116000.caffemodel
mv /content/models/hand/pose_iter_102000.caffemodel /content/openpose/models/hand/pose_iter_102000.caffemodel
mv /content/models/pose/body_25/pose_iter_584000.caffemodel /content/openpose/models/pose/body_25/pose_iter_584000.caffemodel
mv /content/models/pose/coco/pose_iter_440000.caffemodel /content/openpose/models/pose/coco/pose_iter_440000.caffemodel
mv /content/models/pose/mpi/pose_iter_160000.caffemodel /content/openpose/models/pose/mpi/pose_iter_160000.caffemodel
rm -rf models
rm models.zip

In [ ]:
! cd openpose && mkdir build && cd build

In [ ]:
! cd openpose/build && cmake -DUSE_CUDNN=OFF -DBUILD_PYTHON=ON ..

In [ ]:
# ! cd openpose/build && cmake ..

In [ ]:
! cd openpose/build && make -j`nproc`
! cd openpose && mkdir output

[  6%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_crop_layer.cu.o
[  6%] Building NVCC (Device) object src/caffe/CMakeFiles/cuda_compile_1.dir/layers/cuda_compile_1_generated_cudnn_conv_layer.cu.o


In [ ]:
import os
%cd /content/
!rm -rf clothes-virtual-try-on
!git clone https://github.com/practice404/clothes-virtual-try-on.git
os.makedirs("/content/clothes-virtual-try-on/checkpoints")

In [ ]:
!gdown --id 18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6 --output /content/clothes-virtual-try-on/checkpoints/alias_final.pth
!gdown --id 1uDRPY8gh9sHb3UDonq6ZrINqDOd7pmTz --output /content/clothes-virtual-try-on/checkpoints/gmm_final.pth
!gdown --id 1d7lZNLh51Qt5Mi1lXqyi6Asb2ncLrEdC --output /content/clothes-virtual-try-on/checkpoints/seg_final.pth

In [ ]:
!gdown --id 1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx --output /content/clothes-virtual-try-on/cloth_segm_u2net_latest.pth --no-cookies

In [ ]:
%cd /content/
!pip install ninja

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints

In [ ]:
# downloading LIP dataset model
!gdown --id 1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
!mv /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth /content/Self-Correction-Human-Parsing/checkpoints/final.pth

In [ ]:
%cd /content/

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [ ]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

In [ ]:
!conda install --channel conda-forge featuretools --yes

In [ ]:
!pip install opencv-python torchgeometry

In [ ]:
!pip install torchvision

# Welcome to Virtual-Cloth-Assistant

> It'll take some extra time in first execution for setting up and downloading of model weights

In [ ]:
def make_dir():
  os.system("cd /content/ && mkdir inputs && cd inputs && mkdir test && cd test && mkdir cloth cloth-mask image image-parse openpose-img openpose-json")

In [ ]:
from PIL import Image

def run(cloth, model):
  make_dir()
  cloth.save("/content/inputs/test/cloth/cloth.jpg")
  model.save("/content/inputs/test/image/model.jpg")

  # running script to compute the predictions
  os.system("rm -rf /content/output/")
  os.system("python /content/clothes-virtual-try-on/run.py")

  # loading output
  op = os.listdir("/content/output")[0]
  op = Image.open(f"/content/output/{op}")
  return op

In [ ]:
import gradio as gr

with gr.Blocks() as app:
    with gr.Row():
        gr.Markdown("<center><h1> Clothes Virtual Try ON </h1><center>")
    with gr.Row():
        with gr.Column():
            cloth_input = gr.Image(sources=['upload'], type="pil", label="Upload the Cloth Image")
        with gr.Column():
            model_input = gr.Image(sources=['upload'], type="pil", label="Upload the Human Image")
    with gr.Row():
        final_output = gr.Image(sources=['upload'], type="pil", label="Final Prediction")
    with gr.Row():
        submit_button = gr.Button("Submit")

    submit_button.click(fn=run, inputs=[cloth_input, model_input],
                        outputs=[final_output])
app.launch(debug=True, share=True)